# Data Cleaning

## Objectives

*   Encode categorical variables and handle missing data
*   Clean data

## Inputs

* outputs/datasets/collection/HeritageHousing.csv

## Outputs

* Generate a pipeline that performs the data cleaning

## Conclusions

  * Data Cleaning Pipeline
  * Two variables have more than 90% missing values and can be dropped, seven more variables have missing values and those values can be replaced with the variable's median


---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [23]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/Project-heritage-housing-issues'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chdir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/Project-heritage-housing-issues'

---

# Load Collected data

In [4]:
! pip install pandas


[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [5]:
import pandas as pd
df = (pd.read_csv("outputs/datasets/collection/HousePricesRecords.csv"))
df.head(3)

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,...,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,856.0,854.0,3.0,No,706.0,GLQ,150.0,0.0,548.0,RFn,...,65.0,196.0,61.0,5,7,856.0,0.0,2003,2003,208500.0
1,1262.0,0.0,3.0,Gd,978.0,ALQ,284.0,NaN,460.0,RFn,...,80.0,0.0,0.0,8,6,1262.0,NaN,1976,1976,181500.0
2,920.0,866.0,3.0,Mn,486.0,GLQ,434.0,0.0,608.0,RFn,...,68.0,162.0,42.0,5,7,920.0,NaN,2001,2002,223500.0


## Encode object variables:

#### There is four object variables:

**BsmtExposure:** Refers to walkout or garden level walls Gd: Good Exposure; Av: Average Exposure; Mn: Mimimum Exposure; No: No Exposure; None: No Basement

**BsmtFinType1:** Rating of basement finished area GLQ: Good Living Quarters; ALQ: Average Living Quarters; BLQ: Below Average Living Quarters; Rec: Average Rec Room; LwQ: Low Quality; Unf: Unfinshed; None: No Basement

**GarageFinish:** Interior finish of the garage Fin: Finished; RFn: Rough Finished; Unf: Unfinished; None: No Garage

**KitchenQual:** Kitchen quality Ex: Excellent; Gd: Good; TA: Typical/Average; Fa: Fair; Po: Poor

In [6]:
# Dictionary to map the categories of the four object variables to numbers.
# It maps None to zero in contrast to the dictionary used in the HouseSalePrices
# notebook where we discarded None values.
dic = {'BsmtExposure': {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'None': 0}, 'BsmtFinType1': {'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'None': 0}, 'GarageFinish': {'Fin': 3, 'RFn': 2, 'Unf': 1, 'None': 0}, 'KitchenQual': {'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'Po': 0}}
df2=df.copy()
for col in df.columns[df.dtypes=='object'].to_list():
    df2[col] = df2[col].replace(dic[col])
df2.head()

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,...,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,856.0,854.0,3.0,1.0,706.0,6.0,150.0,0.0,548.0,2.0,...,65.0,196.0,61.0,5,7,856.0,0.0,2003,2003,208500.0
1,1262.0,0.0,3.0,4.0,978.0,5.0,284.0,NaN,460.0,2.0,...,80.0,0.0,0.0,8,6,1262.0,NaN,1976,1976,181500.0
2,920.0,866.0,3.0,2.0,486.0,6.0,434.0,0.0,608.0,2.0,...,68.0,162.0,42.0,5,7,920.0,NaN,2001,2002,223500.0
3,961.0,NaN,NaN,1.0,216.0,5.0,540.0,NaN,642.0,1.0,...,60.0,0.0,35.0,5,7,756.0,NaN,1915,1970,140000.0
4,1145.0,NaN,4.0,3.0,655.0,6.0,490.0,0.0,836.0,2.0,...,84.0,350.0,84.0,5,8,1145.0,NaN,2000,2000,250000.0


### Perform the same encoding as above but with a custom transformer that can be used in a pipeline

In [9]:
! pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 37.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 55.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 54.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [10]:
from sklearn.base import BaseEstimator, TransformerMixin
# create a Class variable with a fit and transform method
class MyCustomEncoder(BaseEstimator, TransformerMixin):

  def __init__(self, variables, dic):
    if not isinstance(variables, list): 
      self.variables = [variables]
    else: self.variables = variables
    self.dic = dic

  def fit(self, X, y=None):    
    return self

  def transform(self, X):
    for col in self.variables:
      if X[col].dtype == 'object':
        X[col] = X[col].replace(dic[col])
      else:
        print(f"Warning: {col} data type should be object to use MyCustomEncoder()")
      
    return X

# use the custom encoder in a pipeline
from sklearn.pipeline import Pipeline
pipeline = Pipeline([('custom_encoder', MyCustomEncoder(variables=['BsmtExposure', 'BsmtFinType1', 'GarageFinish', 'KitchenQual'], dic=dic))])

df2 = df.copy()
df2 = pipeline.fit_transform(df2)
df2.head(3)

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,...,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,856.0,854.0,3.0,1.0,706.0,6.0,150.0,0.0,548.0,2.0,...,65.0,196.0,61.0,5,7,856.0,0.0,2003,2003,208500.0
1,1262.0,0.0,3.0,4.0,978.0,5.0,284.0,NaN,460.0,2.0,...,80.0,0.0,0.0,8,6,1262.0,NaN,1976,1976,181500.0
2,920.0,866.0,3.0,2.0,486.0,6.0,434.0,0.0,608.0,2.0,...,68.0,162.0,42.0,5,7,920.0,NaN,2001,2002,223500.0


## Missing data

* Find out which variables have missing data

In [11]:
vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_with_missing_data

['2ndFlrSF',
 'BedroomAbvGr',
 'BsmtExposure',
 'BsmtFinType1',
 'EnclosedPorch',
 'GarageFinish',
 'GarageYrBlt',
 'LotFrontage',
 'MasVnrArea',
 'WoodDeckSF']

#### Explore the variables with missing data

In [16]:
! pip install pandas_profiling


[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [21]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df=df[vars_with_missing_data], minimal=True)
profile.to_notebook_iframe()

ImportError: cannot import name 'TypeAliasType' from 'typing_extensions' (/home/gitpod/.pyenv/versions/3.11.1/lib/python3.11/site-packages/typing_extensions.py)

### Drop variables

* The variables 'EnclosedPorch' and 'WoodDeckSF' have about 90% missing data and can be dropped.

In [ ]:
from feature_engine.selection import DropFeatures
drop_features = DropFeatures(features_to_drop = ['EnclosedPorch', 'WoodDeckSF'])

df_transformed = drop_features.fit_transform(df)
df_transformed.info()

### Replace missing data with median

In [ ]:
vars_with_missing_data

In [ ]:
# Remove 'EnclosedPorch' and 'WoodDeckSF' from vars_with_missing_dat since we drop them
vars_with_missing_data = ['2ndFlrSF', 'BedroomAbvGr', 'BsmtFinType1', 'GarageFinish', 'GarageYrBlt', 'LotFrontage', 'MasVnrArea']
from feature_engine.imputation import MeanMedianImputer

pipeline = Pipeline([
      ('drop_features', DropFeatures(features_to_drop = ['EnclosedPorch', 'WoodDeckSF'])),
      ('custom_encoder', MyCustomEncoder(variables=['BsmtExposure', 'BsmtFinType1', 'GarageFinish', 'KitchenQual'], dic=dic)),
      ('median_imputer',  MeanMedianImputer(imputation_method='median', variables=vars_with_missing_data))
])

df2 = df.copy()
df_transformed = pipeline.fit_transform(df2) 
df_transformed.head(5)   

* Check that there is no missing data

In [ ]:
vars_with_missing_data = df_transformed.columns[df_transformed.isna().sum() > 0].to_list()
vars_with_missing_data

* Print the computed median for each feature

In [ ]:
pipeline['median_imputer'].imputer_dict_

---